In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import pandas as pd
hadiths = pd.read_csv("csv/hadith.csv")

In [11]:
hadiths

,id,arabic,translate_id,narator,hadith_scholar,source,hadith_no,level_of_hadith
0,0,"فِي رِوَايَتِهِ ""إِذَا وَلَدَتِ الأَمَةُ بَعْل...","“Dalam riwayat lain, ""... ketika para al-amatu...",Abu Hurairah,Muslim bin Hajjaj,Shahih Muslim,6,Shahih
1,1,عَنْ يَحْيَى بْنِ يَعْمَرَ، عَنِ ابْنِ عُمَرَ،...,"“Dari Yahya bin Ya'mar dari Ibnu Umar, dari Um...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,4,Shahih
2,2,عن يحيى بن يَعْمَرَ، وَحُمَيْدِ بْنِ عَبْدِ ال...,“Dari Yahya bin Ya'mar dan Humaid bin 'Abdur-R...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,3,Shahih
3,3,عَنْ يَحْيَى بْنِ يَعْمَرَ قَالَ لَمَّا تَكَلّ...,"“Yahya bin Ya'mar berkata: ""Ketika Ma'bad mula...",Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,2,Shahih
4,4,كَانَ أَوَّلَ مَنْ قَالَ فِي الْقَدَرِ بِالْبَ...,“Orang yang pertama kali membahas takdir di Ba...,Abdullah bin Umar,Muslim bin Hajjaj,Shahih Muslim,1,Shahih
...,...,...,...,...,...,...,...,...
778,778,مَن سَمِعَ النِّداءَ فلَم يأتِ فلا صَلاةَ لَه ...,"“Barangsiapa yang mendengar adzan, namun tidak...",Abdullah bin Abbas,Ibnu Hajar Al Asqalani,Bulughul Maram,114,Shahih
779,779,إنَّ لك من الأجرِ على قدْرِ نَصَبِك و نفقتِك,“Sesungguhnya besar pahala sebanding dengan ke...,Aisyah,Muhammad Nashiruddin Al Albani,Shahih Jami' Ash Shaghir,2160,Shahih
780,780,يَوَدُّ أَهلُ العَافِيَةِ يَومَ القِيَامَةِ حِ...,“Orang-orang yang dahulu sehat di dunia kelak ...,Jabir bin Abdillah,Muhammad Nashiruddin Al Albani,Shahih At Tirmidzi,2402,Shahih
781,781,: خطبنا رسول الله صلى الله عليه وسلم، فأقبل ال...,“Rasulullah Shallallahu ‘alaihi wa sallam berk...,Abdullah bin Buraidah,Muhammad Nashiruddin Al Albani,Shahih Abu Daud,1109,Shahih


In [25]:
ids = hadiths["id"].astype(str).tolist()
texts = hadiths["translate_id"].tolist()
metadatas = hadiths.apply(
    lambda x: {
        "id": x["id"],
        "arabic": x["arabic"],
        "translate_id": x["translate_id"],
        "narrator": x["narator"],
        "source": x["source"],
        "hadith_no": x["hadith_no"],
    },
    axis=1,
).tolist()

In [28]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")



In [30]:
from langchain.embeddings.base import Embeddings
from typing import List
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return self.model.encode(documents)

    def embed_query(self, query: str) -> List[float]:
        return self.model.encode([query])[0]

In [31]:
Chroma.from_texts(texts=texts, metadatas=metadatas, embedding=SentenceTransformerEmbeddings("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"))

InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 1536

In [26]:
db_hadith = Chroma.from_texts(texts=texts, metadatas=metadatas, embedding=OpenAIEmbeddings())

In [27]:
query = "Tata cara sholat"
docs = db_hadith.similarity_search(query, k=10)
docs

[Document(id='a90c595f-7921-406a-9882-f08dad2371d4', metadata={'arabic': 'إِذا قَامَ أحدكُم من اللَّيْل فليفتتح صلَاته بِرَكْعَتَيْنِ خفيفتين', 'hadith_no': '768', 'id': 118, 'narrator': 'Abu Hurairah', 'source': 'Shahih Muslim', 'translate_id': '“Jika kalian bangun shalat malam, mulailah dengan shalat dua rakaat yang ringan”'}, page_content='“Jika kalian bangun shalat malam, mulailah dengan shalat dua rakaat yang ringan”'),
 Document(id='e07843d9-736d-490f-8e6e-a82c7a0748b6', metadata={'arabic': 'إِنِّي لَأَدْخُلُ فِي الصَّلاَةِ وَأَنَا أُرِيدُ إِطَالَتَهَا، فَأَسْمَعُ بُكَاءَ الصَّبِيِّ، فَأَتَجَوَّزُ فِي صَلاَتِي مِمَّا أَعْلَمُ مِنْ شِدَّةِ وَجْدِ أُمِّهِ مِنْ بُكَائِهِ', 'hadith_no': '709', 'id': 660, 'narrator': 'Anas bin Malik', 'source': 'Shahih Al Bukhari', 'translate_id': '“Aku pernah mengimami shalat dan aku berniat untuk memperpanjang shalat tersebut. Namun aku mendengar tangisan anak kecil. Maka aku ringkas shalat tersebut karena aku memahami betapa berat perasaan ibunya k